# Generating valdiation set's trials

## from validation set

In [1]:
import pandas as pd

In [8]:
val_df = pd.read_csv("/dataset/SV_sets/voxceleb1/dataframes/si_val.csv")
idx2key = val_df.id.to_dict()
key2idx = {v:k for k,v in idx2key.items()}

In [10]:
def diff_session(pair):
    a, b = pair
    if a[8:19] != b[8:19]:
        return True
    else:
        return False

In [13]:
# positive pair
import itertools
pos_trials = []
pos_len = dict()
val_spks = val_df.spk.unique().tolist()
for spk in val_spks:
    utters = val_df[val_df.spk == spk].id
    every_pos_pair = list(filter(diff_session, itertools.combinations(utters, 2)))
    pos_trials += every_pos_pair
    pos_len[spk] = len(every_pos_pair) 

In [33]:
# negative m pair
import itertools
import random
neg_trials = []
for spk in val_spks:
    utters = val_df[ (val_df.spk == spk)].id
    neg_targets = val_df[(val_df.spk != spk)].sample(n=pos_len[spk]//len(utters)+1).id
    neg_trials += list(itertools.product(utters, neg_targets))

In [53]:
import numpy as np

pairs = pos_trials + neg_trials
enr_ids, test_ids = zip(*pairs)
labels = np.ones(len(pos_trials), dtype=int).tolist() + \
            np.zeros(len(neg_trials), dtype=int).tolist()

assert len(pairs) == len(labels)

In [55]:
idx_pairs = [(key2idx[enr_id], key2idx[test_id])  for enr_id, test_id in pairs]
enr_idxs, test_idxs = zip(*idx_pairs)

In [66]:
val_trial_df = pd.DataFrame({'label':labels, 'enroll_id':enr_ids, 'test_id':test_ids, 
             'enroll_idx':enr_idxs, 'test_idx':test_idxs})
val_trial_df.to_csv("/dataset/SV_sets/voxceleb1/val_trial.csv", index=False)

## from ood set

In [109]:
val_df = pd.read_csv("/dataset/SV_sets/voxceleb2/dataframes/voxc2_si.csv")
val_spks = val_df.spk.value_counts().index[-1000:]
val_df = val_df[val_df.spk.isin(val_spks)].reset_index()
idx2key = val_df.id.to_dict()
key2idx = {v:k for k,v in idx2key.items()}

In [110]:
val_df.to_csv("/dataset/SV_sets/voxceleb1/voxc2_val.csv", index=False)

In [111]:
def diff_session(pair):
    a, b = pair
    if a[8:19] != b[8:19]:
        return True
    else:
        return False

In [112]:
# positive pair
import itertools
import random
pos_trials = []
pos_len = dict()
for spk in val_spks:
    utters = val_df[val_df.spk == spk].id
    every_pos_pair = random.choices(list(filter(diff_session, itertools.combinations(utters, 2))), k=20)
    pos_trials += every_pos_pair
    pos_len[spk] = len(every_pos_pair) 

In [114]:
# negative m pair
import itertools
import random
neg_trials = []
for spk in val_spks:
    utters = val_df[ (val_df.spk == spk)].id
    neg_targets = val_df[(val_df.spk != spk)].sample(n=pos_len[spk]//len(utters)+1).id
    neg_trials += list(itertools.product(utters, neg_targets))

In [115]:
import numpy as np

pairs = pos_trials + neg_trials
enr_ids, test_ids = zip(*pairs)
labels = np.ones(len(pos_trials), dtype=int).tolist() + \
            np.zeros(len(neg_trials), dtype=int).tolist()

assert len(pairs) == len(labels)

In [121]:
idx_pairs = [(key2idx[enr_id], key2idx[test_id])  for enr_id, test_id in pairs]
enr_idxs, test_idxs = zip(*idx_pairs)

In [122]:
val_trial_df = pd.DataFrame({'label':labels, 'enroll_id':enr_ids, 'test_id':test_ids, 
             'enroll_idx':enr_idxs, 'test_idx':test_idxs})
val_trial_df.to_csv("/dataset/SV_sets/voxceleb1/voxc2_val_trial.csv", index=False)